<a href="https://colab.research.google.com/github/Dvdbijl/CO2-Emissions---Final-Assignment/blob/main/CO2_Emissions_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vraag 1


In [46]:
import pandas as pd
import numpy as np
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
df_emmissions = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/co-emissions-per-capita.csv")
df_gdp = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/gdp-per-capita-penn-world-table.csv")
df_defor = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/annual-deforestation.csv")
df_meat = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/daily-meat-consumption-per-person.csv")
df_energy = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/share-energy-source-sub.csv")

In [ ]:
print(df_emmissions.head)
print(df_gdp.head)
print(df_defor.head)
print(df_meat.head)
print(df_energy.head)

In [48]:
#Renaming the columns to make it better readable
df_emmissions.rename(columns={'Annual CO₂ emissions (per capita)': "CO2 Emissions"}, inplace=True)
df_gdp.rename(columns={'GDP per capita (output, multiple price benchmarks)': "GDP"}, inplace=True)
df_meat.rename(columns={'Meat, total | 00002943 || Food available for consumption | 0645pc || kilograms per year per capita': "Meat Consumption"}, inplace=True)
df_energy.rename(columns={'Oil (% equivalent primary energy)': "Oil"}, inplace=True)
df_energy.rename(columns={'Coal (% equivalent primary energy)': "Coal"}, inplace=True)
df_energy.rename(columns={'Solar (% equivalent primary energy)': "Solar"}, inplace=True)
df_energy.rename(columns={'Nuclear (% equivalent primary energy)': "Nuclear"}, inplace=True)
df_energy.rename(columns={'Hydro (% equivalent primary energy)': "Hydro"}, inplace=True)
df_energy.rename(columns={'Wind (% equivalent primary energy)': "Wind"}, inplace=True)
df_energy.rename(columns={'Gas (% equivalent primary energy)': "Gas"}, inplace=True)
df_energy.rename(columns={'Other renewables (% equivalent primary energy)': "Other renewables"}, inplace=True)


In [49]:
#Dropping the column Code
df_emmissions.drop(['Code'], axis=1, inplace=True)
df_gdp.drop(['Code'], axis=1, inplace=True)
df_defor.drop(['Code'], axis=1, inplace=True)
df_meat.drop(['Code'], axis=1, inplace=True)
df_energy.drop(['Code'], axis=1, inplace=True)

In [50]:
#Merging the dataframes on the column Entity and Year
data_frames = [df_emmissions, df_gdp, df_defor, df_meat, df_energy]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Entity', 'Year'], how='outer'), data_frames)
#Removing the NaN's and dropping the Year column
df_merged.dropna(inplace=True)
df_merged.drop(['Year'], axis=1, inplace=True)

In [ ]:
print(df_merged.corr())

In [ ]:
plt.figure(figsize=(16, 6))
#Making a mask so that the upper values of the triangle are invincible
mask = np.triu(np.ones_like(df_merged.corr()))
heatmap = sns.heatmap(df_merged.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

In [ ]:
plt.figure(figsize=(8, 12))
#Making a heatmap for just CO2 Emissions in descending manner
heatmap = sns.heatmap(df_merged.corr()[['CO2 Emissions']].sort_values(by='CO2 Emissions', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with CO2 Emissions', fontdict={'fontsize':18}, pad=16);

#Vraag 2

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
%load_ext google.colab.data_table

In [76]:
df_pop = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/population-and-demography.csv")
df_annualco2 = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/annual-co2-emissions-per-country.csv")

In [ ]:
print(df_pop.head)
print(df_annualco2)

In [77]:
#Renaming the columns and dropping the Code column
df_pop.rename(columns={'Country name': "Entity"}, inplace=True)
df_annualco2.rename(columns={'Annual CO₂ emissions': "CO2"}, inplace=True)
df_annualco2.drop(['Code'], axis=1, inplace=True)

In [78]:
#Merging the dataframes on the column Entity and Year
data_frames = [df_pop, df_annualco2]
df = reduce(lambda  left,right: pd.merge(left,right,on=['Entity', 'Year'], how='outer'), data_frames)
#dropping the NaN's
df.dropna(inplace=True)

In [79]:
#Dropping the columns between Population and CO2 column
df.drop(df.columns[3:24], axis=1, inplace=True)

In [ ]:
display(df)

In [80]:
#Calculating CO2 per capita and making a new column with the results
df['CO2 Capita'] = df.CO2 / df.Population

In [ ]:
display(df)

In [82]:
#Calculating the Percentage Change for every column
df['PC Population'] = df.groupby('Entity')['Population'].pct_change() * 100
df['PC CO2'] = df.groupby('Entity')['CO2'].pct_change() * 100
df['PC CO2 Capita'] = df.groupby('Entity')['CO2 Capita'].pct_change() * 100
#Dropping NaN's
df.dropna(inplace=True)

In [ ]:
display(df)

In [ ]:
grouped_data = df.groupby('Entity')
#Define a function to get the results for the last decade
def calculate_sum_change_last_10_years(group):
    # Sort the data by 'Year' within each group
    group = group.sort_values('Year')

    #Calculate the difference between the Percentage Change CO2 and the Percentage Change CO2 per Capita
    group['CO2 Change'] = group['PC CO2'] - group['PC CO2 Capita']

    #Get the CO2 Change for the last 10 years
    last_10_years = group['CO2 Change'].tail(10)

    #Calculate the sum of changes in CO2 Change over the last 10 years
    sum_change_last_10_years = last_10_years.diff().sum()

    #Assign the sum of changes to all rows of the last 10 years
    group.loc[group['Year'] >= group['Year'].max() - 9, 'Change Last Decade'] = sum_change_last_10_years

    return group

#Apply the function to each group
result_new = grouped_data.apply(calculate_sum_change_last_10_years)


In [ ]:
display(result_new)

In [ ]:
#Showing the data of the year 2021
df_filtered = result_new.query('Year >= 2021')
display(df_filtered)

In [ ]:
#Dataframe with just Enity and Change Last Decade column
plot_df = df_filtered[['Entity', 'Change Last Decade']].copy()
display(plot_df)

In [ ]:
#Sort the dataframe by Change Last Decade column in ascending order
plot_df = plot_df.sort_values(by='Change Last Decade', ascending=True)

#Select the top 10 rows
top_10 = plot_df.head(10)

display(top_10)

In [ ]:
#Use Entity as index
top_10.set_index('Entity', inplace=True)
#Change negative numbers to positive
top_10['Change Last Decade Positive'] = top_10['Change Last Decade'].abs()
#Dropping Change Last Decade column
top_10.drop(columns=['Change Last Decade'], inplace=True)

In [ ]:
#Plotting horizontal bar with the results
ax = top_10.plot.barh(width=0.7, fontsize='14', figsize=(11,5));

plt.xlabel('Percentage Change (%)')
plt.ylabel('Country')
plt.title('Top 10 Countries in decreasing CO2 output in last decade')
plt.grid(axis='x')
plt.legend([])

for bar in ax.patches:
    ax.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.2f}',
             ha='left', va='center')

#Vraag 3

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%load_ext google.colab.data_table

In [87]:
df = pd.read_csv("https://github.com/Dvdbijl/CO2-Emissions---Final-Assignment/raw/main/levelized-cost-of-energy.csv")

In [ ]:
#Drop NaN's and the columns Code and Entity
df.dropna(inplace=True)
df.drop(['Code', 'Entity'], axis=1, inplace=True)
display(df)

In [89]:
#Removing levelized cost of energy from every column
df.columns = df.columns.str.replace(' levelized cost of energy', '')

In [90]:
# Melt the DataFrame to have one row per (Year, Price_per_MWh, Energy resource) combination
melted_df = pd.melt(df, id_vars=['Year'], var_name='Energy Resource', value_name='Price per kWh')

In [91]:
#Preparing data for linear regression
X = melted_df[['Year']]  #independent variable
y = melted_df['Price per kWh']  #dependent variable

In [92]:
#Fit a linear regression model for each energy resource
models = {}
for resource in melted_df['Energy Resource'].unique():
    resource_data = melted_df[melted_df['Energy Resource'] == resource]
    model = LinearRegression()
    model.fit(resource_data[['Year']], resource_data['Price per kWh'])
    models[resource] = model

In [ ]:
#Predict future prices for the energy resources
future_years = np.array([[2022], [2023], [2024], [2025], [2026]])
future_prices = {}
for resource, model in models.items():
    future_prices[resource] = model.predict(future_years)

In [ ]:
#Visualize the results
plt.figure(figsize=(10, 6))
for resource, model in models.items():
    resource_data = melted_df[melted_df['Energy Resource'] == resource]
    plt.plot(resource_data['Year'], model.predict(resource_data[['Year']]), label=resource)
    plt.scatter(future_years, future_prices[resource])
plt.xlabel('Year')
plt.ylabel('Price per kWh ($)')
plt.title('Non-fossil fuel energy prices over time with Linear Regression')
plt.legend()
plt.grid(True)
plt.show()